# TP : Traitement Automatisé du Langage avec Textblob et Tweepy

## Analyse de sentiments et traduction automatique

Afin d'obtenir facilement un dataset varié, nous allons récupérer quelques series de Tweets à traiter. Nous utiliserons l'API Tweepy et sauvegarderons un dataset au format json.

### I) Tweepy

    Utiliser Tweepy nécéssite de créer un compte twitter et d'enregistrer une application. De nombreux tutos sont disponibles sur internet, aussi ce tutoriel considère que les étapes préliminaires ont été éffectuées

#### A) Préparation de Tweepy

On commence par effectuer les imports dont nous aurons besoin concernant Tweepy :

In [64]:
import tweepy
import json
import codecs
import time

Il nous faut enssuite renseigner les informations d'accès à l'API Twitter :

(Je vous laisse le soin de rentrer vos propres clefs)

In [65]:
api_key = "YOUR SECRET KEY"
api_key_secret = "YOUR SECRET KEY"
access_token = "YOUR ACCES TOKEN"
access_token_secret = "YOUR ACCES TOKEN"

La fonction OAuthHandler du paquet tweepy est utilisée avec la clef d'API et sa clef secrete pour obtenir l'objet OAuth.

On utilise ensuite la fonction <em>set_access_token</em> pour renseigner l'access_token et son secret.

Et enfin on utilise la fonction <em>API</em> de tweepy pour se connecter et obtenir un objet API.

In [66]:
# Authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

# Retrieving API
api = tweepy.API(auth)

Voici comment vérifier les informations et droit de l'utilisateur avec lequel nous nous sommes connecté

In [67]:
user = api.verify_credentials()
print(user.screen_name)

OsamTek


On peut itérer à travers la timeline de l'utilisateur avec la méthode <em>user_timeline</em>. On spécifie le nom de l'utilisateur cible grace à l'argument <em>screen_name</em>. L'argument <em>count</em> permet de spécifier le nombre de tweets a récupérer. Le nombre maximum de tweets par requête est de 208 au moment où j'écrit ces lignes.

Voici un petit exemple de ce que l'on peut récupérer au format json : 

In [68]:
# Get statuses
for status in api.user_timeline(screen_name="realDonaldTrump", count=1):
    print(status)


Status(_api=<tweepy.api.API object at 0x0000014F41A58CF8>, _json={'created_at': 'Mon Mar 16 15:00:41 +0000 2020', 'id': 1239567267312152576, 'id_str': '1239567267312152576', 'text': 'Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We w… https://t.co/7zIZw2Ykig', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/7zIZw2Ykig', 'expanded_url': 'https://twitter.com/i/web/status/1239567267312152576', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 25073877, 'id_str': '25073877', 'name': 'Donald J. Trump', 'screen_name': 'realDonaldTrump', 'location': 'Washington, DC', 'descri

Et voici comment séléctionner les champs qui nous intéressent :

In [69]:
# Get statuses
for status in api.user_timeline(screen_name="realDonaldTrump", count=1):
    print(status.created_at)
    print(status.user.name)
    print(status.text)

2020-03-16 15:00:41
Donald J. Trump
Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We w… https://t.co/7zIZw2Ykig


Pour information, le champs texte est un champs tronqué. Pour obtenir le texte en entier il faut rajouter ce paramettre : <em>tweet_mode='extended'</em> et changer le nom de champs "text" qui n'existe plus, à "full_text".

Voici le code :

In [70]:
# Get statuses
for status in api.user_timeline(screen_name="realDonaldTrump", count=1, tweet_mode='extended'):
    print(status.created_at)
    print(status.user.name)
    print(status.full_text)

2020-03-16 15:00:41
Donald J. Trump
Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We will end up being stronger than ever before!


#### B) Preparation et sauvegarde des données

Pour des raison de reliabilitée des tests, il est utile de sauvegarder les tweets qui nous servirons de dataset.

Il serait possible de sauvegarder directement l'enssemble des tweets récupérés avec cette commande :

In [71]:
json.dump(status._json, codecs.open(u"dataset.json", 'wb', encoding='utf-8'))
# json.dump(status._json, codecs.open(u"dataset.p", 'wb', encoding='utf-8'))

Mais il me semble plus intéréssant de formatter les entrées pour ne contenir que le contenu qui pourrait nous intérésser.

déterminons donc les champs qui pourrait nous intéresser :
- created_at : Contient la date de création du tweet.
- id : contient l'identifiant unique du tweet.
- full_text : Contient le texte complet du tweet
- display_text_range : contient les informations de taille du texte.
- user : Contient les informations de l'utilisateur. Peut être reformaté pour être plus lisible :
    - user.id
    - user.name
    - user.screen_name
    - user.location
    - user.description
    - user.followers_count
    - user.friends_count
    - user.listed_count
    - user.created_at
    - user.statuses_count
- is_quoted_status : Indique si le tweet est une réponse à un autre tweet.
- quoted_status_id : Contient l'identifiant du tweet quoté.



### II) Utiliser Textblob

#### A) Découvrir Textblob

Nous allons commencer par explorer de manière basique quelques fonctions de Textblob :

Premièrement, n'oublions pas d'effectuer les imports :

In [72]:
from textblob import TextBlob

Nous allons donc récupérer quelques tweets (5) et n'en conserver que le texte :

In [73]:
# Get statuses
status = api.user_timeline(screen_name="realDonaldTrump", count=5, tweet_mode='extended')
# print(status)
print(status[0]._json)
#     print(status.created_at)
#     print(status.user.name)
#     print(status.full_text)

{'created_at': 'Mon Mar 16 15:00:41 +0000 2020', 'id': 1239567267312152576, 'id_str': '1239567267312152576', 'full_text': 'Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We will end up being stronger than ever before!', 'truncated': False, 'display_text_range': [0, 158], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 25073877, 'id_str': '25073877', 'name': 'Donald J. Trump', 'screen_name': 'realDonaldTrump', 'location': 'Washington, DC', 'description': '45th President of the United States of America🇺🇸', 'url': 'https://t.co/OMxB0x7xC5', 'entities': {'url': {'urls': [{'url': 'https://t.co/OMxB0x7xC5', 'expanded_url': 'http://w

Pour détailler chaques étapes, nous allons créer une variable temporaire ne contenant qu'une entrée :

In [74]:
entry = status[0]
print(entry.full_text)

Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We will end up being stronger than ever before!


Pour dire a textblob d'analyser le texte voici la commande :

In [75]:
#Analyse de tweet
tweet = TextBlob(entry.full_text)


Depuis cet objet nous pouvons accéder à l'analyse de sentiments :

In [76]:
tweet.sentiment

Sentiment(polarity=0.5145833333333334, subjectivity=0.7229166666666667)

Ainssi qu'à la détection de la langue :

In [77]:
tweet.detect_language()

'en'

Et nous pouvons aussi traduire le texte :

In [78]:
tweet.translate(from_lang="en", to="fr")

TextBlob("Tout le monde est si bien unifié et travaille si dur. C'est une belle chose à voir. Ils aiment notre grand pays. Nous finirons par être plus forts que jamais!")

Il est aussi possible de Tokeniser la phrase comme ceci :

In [79]:
print("Tokens : {}".format(tweet.words))

Tokens : ['Everybody', 'is', 'so', 'well', 'unified', 'and', 'working', 'so', 'hard', 'It', 'is', 'a', 'beautiful', 'thing', 'to', 'see', 'They', 'love', 'our', 'great', 'Country', 'We', 'will', 'end', 'up', 'being', 'stronger', 'than', 'ever', 'before']


<strong>!! Attention</strong>, il est peut-être nécéssaire de télécharger un complément (le corpus semble-t-il de NLTK..) la commande devrait être suggérée dans le message d'erreur. Il suffit alors d'ouvrir un terminal et de l'executer

Il est temps maintenant de rassembler ces possibilitées au sein d'une boucle et d'améliorer un peu l'affichage :

In [80]:
for entry in status :
    tweet = TextBlob(entry.full_text)
    print(tweet)
    print("Tokens : {}".format(tweet.words))
    print("Polarity %.2f, Subjectivity %.2f" % (tweet.sentiment.polarity, tweet.sentiment.subjectivity))
    print("Language : {}".format(tweet.detect_language()))
    try:
        print("French : {}".format(tweet.translate(from_lang="en-US", to='fr')))
    except textblob.exceptions.NotTranslated:
        pass
    # end try
    print()

Everybody is so well unified and working so hard. It is a beautiful thing to see. They love our great Country. We will end up being stronger than ever before!
Tokens : ['Everybody', 'is', 'so', 'well', 'unified', 'and', 'working', 'so', 'hard', 'It', 'is', 'a', 'beautiful', 'thing', 'to', 'see', 'They', 'love', 'our', 'great', 'Country', 'We', 'will', 'end', 'up', 'being', 'stronger', 'than', 'ever', 'before']
Polarity 0.51, Subjectivity 0.72
Language : en
French : Tout le monde est si bien unifié et travaille si dur. C'est une belle chose à voir. Ils aiment notre grand pays. Nous finirons par être plus forts que jamais!

God Bless the USA!
Tokens : ['God', 'Bless', 'the', 'USA']
Polarity 0.00, Subjectivity 0.00
Language : en
French : Que Dieu bénisse les USA!

RT @parscale: Joe Biden is a train wreck on illegal immigration and would harm national security.

He’d have ZERO deportations for the firs…
Tokens : ['RT', 'parscale', 'Joe', 'Biden', 'is', 'a', 'train', 'wreck', 'on', 'illegal